# Recalibrating the ringdown attachment frequency for ENIGMA

 - The idea here is to use an MCMC algorithm to find the best parameters that make ENIGMA agree with SEOBNRv4 in the regions of parameter space that overlap for them
 - We will use the emcee package for this
 - We need a few components for MCMC:
    - A way to generate ENIGMA for a given set of parameters **$\theta$**


**Define**:

 1. Fix $\{m_1, m_2\}$
 1. $\theta = \{\omega_\mathrm{attach}, \,\mathrm{PN\, order}\}$
 1. ndim = len(\theta) = 2
 1. log_prob $\propto \langle h_1, h_2 \rangle$ 

**Choose internal parameters**:

 1. PSD $\in$ [**aLIGOZeroDetHighPower**, **Flat**]
 2. low frequency cutoff $\in [15, 20, 30]Hz$
 2. sample rate = $2048$
 2. masses $(m_1, m_2) \in [40, 60]M_\odot$
 2. attachment time $t_\mathrm{attach} \in $ **???**
 2. attachment time window $dt_\mathrm{attach} \in $ **???**

**Algorithm**:

* Sample $\theta$:
 1. PNO $\in [6, 7, 8, 9, 10, 11, 12]$
 1. $\omega_\mathrm{attach} \in [0.01, 0.1]$
* Do:
 1. Compute $h_1 \equiv$ h_ENIGMA$\left(m_1, m_2, \theta; t\right)$
 1. Compute $h_2 \equiv$ h_EOB$\left(m_1, m_2, \theta; t\right)$
 1. Compute log_prob = $\propto \langle h_1, h_2 \rangle$

**Code plan**:

* `gwnr_enigma_plan_grid_and_make_dag`
 - Inputs:
   - mass1 range, num_points
   - mass2 range, num_points
   - f_lower choices
   - psd choices
   - num_unique_mcmc_per_job
 - Meat:
   - Generate a hypercubic lattice over {mass1, mass2, f_lower, psd_choices}
   - 
 - Output:
   - DAG
* `gwnr_enigma_sample_parameters`
 - Inputs:
   - Fixed values of mass1
   - Fixed values of mass2
   - Fixed values of f_lower
   - Fixed values of psd
 - Meat:
   - Iterate over {mass1, mass2, f_lower, psd_choices}
   - MCMC over $\{\omega_\mathrm{attach}, \mathrm{PNO}\}$
 - Output:
   - Text file with columns:
     - [1] mass-ratio
     - [2] total-mass
     - [3] mass1
     - [4] mass2
     - [5] ecc value
     - [6] PN order
     - [7] omega_attach
     - [8] overlap
     - [9] Re[unphase-maximized overlap]
     - [10] Im[unphase-maximized overlap]
     - [11] norm1
     - [12] norm2

In [1]:
import os
import logging
import numpy as np
import h5py
import emcee

In [2]:
from pycbc.waveform import get_td_waveform, get_fd_waveform
from pycbc.psd import from_string
from pycbc.filter import match

In [3]:
import sys

sys.path.append(os.path.join(os.environ['HOME'], 'src/gwnrtools'))

In [4]:
import gwnr
from gwnr.Utils import make_padded_frequency_series

In [5]:
logging.getLogger().setLevel(logging.INFO)

In [6]:
run_directory = '/home/prayush/TESTME'

## `gwnr_enigma_plan_grid_and_make_dag`

**Code plan**:

* `gwnr_enigma_plan_grid_and_make_dag`
 - Inputs:
   - mass1 range, num_points
   - mass2 range, num_points
   - f_lower choices
   - psd choices
   - num_unique_mcmc_per_job
 - Meat:
   - Generate a hypercubic lattice over {mass1, mass2, f_lower, psd_choices}
   - 
 - Output:
   - DAG

In [7]:
!gwnr_enigma_plan_calib_grid_and_make_dag --help

usage: /home/prayush/src/GWNRTools/bin//gwnrtools_enigma_plan_calib_grid_and_make_dag [--options]

Setup workflow to perform calibration of ENIGMA model using another BBH model

optional arguments:
  -h, --help            show this help message and exit
  --version             Prints version information.
  --verbose             Print logging messages.
  --min-mass1 MIN_MASS1
                        Lower limit for mass of bigger BH.
  --max-mass1 MAX_MASS1
                        Upper limit for mass of bigger BH.
  --min-mass2 MIN_MASS2
                        Lower limit for mass of smaller BH.
  --max-mass2 MAX_MASS2
                        Upper limit for mass of smaller BH.
  --min-q MIN_Q         Lower limit for mass of smaller BH.
  --max-q MAX_Q         Upper limit for mass of smaller BH.
  --grid-spacing-mass1 GRID_SPACING_MASS1
                        Step size for sampling mass of bigger BH.
  --grid-spacing-mass2 GRID_SPACING_MASS2
                        Step size for samp

In [13]:
!gwnr_enigma_plan_calib_grid_and_make_dag --verbose \
    --min-mass1=3 \
    --max-mass1=100 \
    --grid-spacing-mass1 1 \
    --min-mass2=3 \
    --max-mass2=100 \
    --grid-spacing-mass2 1 \
    --min-q=1 \
    --max-q=10 \
    --min-omega-attach=0.01 \
    --max-omega-attach=0.1 \
    --choices-pn-order="6,7,8,9,10,11,12" \
    --choices-f-lower="15,30" \
    --signal-approx='SEOBNRv4_ROM' \
    --time-length=64 \
    --num-samplers=32 \
    --num-mcmc-steps 50 \
    --num-calcs-per-job 10 \
    --run-dir=/home/prayush/TESTME \
    --output-prefix=results/matches_vs_

INFO:root:Going to setup run with:
INFO:root:Mass1 $\in$ [3.0, 100.0]Msun
INFO:root:Mass2 $\in$ [3.0, 100.0]Msun
INFO:root:Mass-ratio $\in$ [1.0, 10.0]Msun
INFO:root:... with spacing: 1.0, 1.0
INFO:root:Will sample lower freq cutoff from: [15. 30.]
INFO:root:Will sample PSD from: ['aLIGOZeroDetHighPower']
INFO:root:Will setup 10 calculations per job
INFO:root: Setting up the workflow in /home/prayush/TESTME now ...

INFO:root:.. directories made.
INFO:root: .. parameter grid constructed.
INFO:root: .. condor submission file written.
INFO:root: .. DAG written.
INFO:root:All Done in 2.70973086357 seconds


## `gwnr_enigma_sample_parameters`

**Code plan**

* `gwnr_enigma_sample_parameters`:
 - Inputs:
   - Parameter file
 - Meat:
   - Iterate over {mass1, mass2, f_lower, psd_choices}
   - MCMC over $\{\omega_\mathrm{attach}, \mathrm{PNO}\}$
 - Output:
   - Text file with columns:
     - [1] mass-ratio
     - [2] total-mass
     - [3] mass1
     - [4] mass2
     - [5] ecc value
     - [6] PN order
     - [7] omega_attach
     - [8] overlap
     - [9] Re[unphase-maximized overlap]
     - [10] Im[unphase-maximized overlap]
     - [11] norm1
     - [12] norm2

In [7]:
os.chdir(run_directory)

In [8]:
!gwnr_enigma_sample_calib_parameters --help

usage: /home/prayush/src/GWNRTools/bin//gwnrtools_enigma_sample_calib_parameters [--options]

Perform calibration runs for ENIGMA model using another BBH model

optional arguments:
  -h, --help            show this help message and exit
  --version             Prints version information.
  --verbose             Print logging messages.
  --job-id JOB_ID
  --param-file PARAM_FILE
                        Input parameter file
  --min-omega-attach MIN_OMEGA_ATTACH
                        Lower limit for dimless attachment freq.
  --max-omega-attach MAX_OMEGA_ATTACH
                        Upper limit for dimless attachment freq.
  --choices-pn-order CHOICES_PN_ORDER
                        Comma-separated list of PN orders to try
  --signal-approx SIGNAL_APPROX
                        Signal approximant to test against (FD)
  --num-samplers NUM_SAMPLERS
                        No of MCMC walkers
  --num-mcmc-steps NUM_MCMC_STEPS
                        No of MCMC steps per walker
  --sample

In [9]:
!ls input/parameters_000*

input/parameters_000000.hdf  input/parameters_000500.hdf
input/parameters_000100.hdf  input/parameters_000600.hdf
input/parameters_000200.hdf  input/parameters_000700.hdf
input/parameters_000300.hdf  input/parameters_000800.hdf
input/parameters_000400.hdf  input/parameters_000900.hdf


In [11]:
!gwnr_enigma_sample_calib_parameters --verbose \
    --job-id "000200" \
    --param-file input/parameters_000200.hdf \
    --num-samplers 32 \
    --num-mcmc-steps 10 \
    --sample-rate 2048 \
    --output-prefix results/matches_vs_

INFO:root:Starting job: 000200
INFO:root:Will read from: input/parameters_000200.hdf
INFO:root:Will compute matches against SEOBNRv4_ROM
INFO:root:Will sample omega_att: [0.01, 0.1]
INFO:root:Will sample PN order from: 6,7,8,9,10,11,12
INFO:root:Will take 10 MCMC steps per sampler
INFO:root:Will use 32 samplers
INFO:root:Will write output with prefix: results/matches_vs_SEOBNRv4_ROM
INFO:root:Will filter at 2048Hz with maxT = 32secs
INFO:root:.. input parameters read.
INFO:root: ... starting MCMC for setting 0: m1=16.00, m2=12.00, f_low=15
(320, 8)
INFO:root: ... starting MCMC for setting 1: m1=16.00, m2=12.00, f_low=30
XLAL Error - x_model_eccbbh_waveform (LALSimENIGMA.c:673): initial phi_dot > omega_attach: 0.0129981 > 0.0104406
XLAL Error - x_model_eccbbh_waveform (LALSimENIGMA.c:673): Invalid argument
XLAL Error - XLALSimInspiralENIGMA (LALSimENIGMA.c:1133): Internal function call failed: Invalid argument
ERROR:root:Could not generate ENIGMA wave..
XLAL Error - x_model_eccbbh_wavef